In [ ]:
import csv
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import ui
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [ ]:
def __scroll_down_page(): #Set a function to scroll down the current page
    current_scroll_position, new_height = 0, 1 #Set current scroll position as 0 and the position after scrolling as 1
    while current_scroll_position <= new_height:
        current_scroll_position += 40
        #Executes JavaScript and insert the formated current scroll position inside the string's placeholder to perform scrolling down action
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position)) 
        # Get new scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")


In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, executable_path="C:\ProgramData\Anaconda3\Lib\site-packages\jupyterlab\chromedriver.exe")

In [ ]:
driver.get('https://en.tripadvisor.com.hk/Hotels-g294217-oa150-zfc3,4,5-Hong_Kong-Hotels.html')
driver.maximize_window()
time.sleep(5)
driver.find_element_by_xpath('//div[@class="fylXE"]/div[contains(text(),"5")]').click()
time.sleep(2)
driver.find_element_by_xpath('//div[@class="fylXE"]/div[contains(text(),"6")]').click()
time.sleep(2)
driver.find_element_by_xpath('//button[contains(text(),"Update")]').click()
time.sleep(2)
driver.find_element_by_css_selector('span>div.cDAjs').click() #click"Sort by" box
time.sleep(2)
driver.find_element_by_xpath('//div[contains(text(),"Traveller Ranked")]/..').click()
time.sleep(3)


In [ ]:
hotelname = []
urls = []
check = 0
outer_break = False
for i in range(30):
    try:
        time.sleep(10)
        __scroll_down_page()
        #Executes javascript to print the outerHTMl of the current webpage for Beautiful Soup
        data2 = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(data2, 'html.parser')
        for link in soup.find_all('div', {'class': 'listing_title'}):
            sponsored = link.find('span', {'class': 'sponsored_v2'})
            print(sponsored)
            #Skip the sponsored hotels that show in the middle of the webpages
            if sponsored:
                continue
            #If there are 300 links of the hotels collected, the process of scraping will end
            if check == 300:
                outer_break = True
                break
            b = link.find('a', href=True)
            hotel_url = b['href']
            urls.append(hotel_url)
            hotelname.append(link.find('a', {'class': 'property_title'}).text)
            check = check + 1
        try:
            ui.WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//a[contains(text(),"Next")]'))).click()
        except:
            time.sleep(10)
            ui.WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//a[contains(text(),"Next")]'))).click()
    except:
        driver.refresh()

    time.sleep(10)
    if outer_break:
        break
driver.close()

In [ ]:
data= {
    'Hotel Name': hotelname,
    'Hotel URL': urls
}

In [ ]:
dataforcomment={
    'Hotel Name': hotelname[:10],
    'Hotel URL': urls[:10]
}

In [ ]:
import pandas as pd
Top300hotel = pd.DataFrame.from_dict(data)
Top10hotel = pd.DataFrame.from_dict(dataforcomment)

In [ ]:
Top300hotel.to_csv('Top_300.csv')

In [ ]:
Top10hotel.to_csv('Top_10.csv')